In [38]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pybaseball as mlb
import mlbgame

import preprocess

%load_ext autoreload
%autoreload 2
from jesse import jesse
jesse.SetOptions()

today = datetime.today().strftime('%Y-%m-%d')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/downing.jesse/Projects/MLB_statTRAX/python


In [42]:
test = preprocess.get_mlb_dates(2018)

In [43]:
end_dt = datetime.strptime(test.second_half_end,'%Y-%m-%d')

In [45]:
test.allstar_break_days

['2018-07-16', '2018-07-17', '2018-07-18']

In [ ]:
a.set_mlb_dates()
a.set_endpoints()
a.set_all_star_break_days()
a.set_season_days_so_far()
a.get_season_days_so_far()

In [ ]:
def get_current_season():
    return datetime.today().year

In [ ]:
def get_schedule_dates(season):
    mlb_dates = mlbgame.important_dates(year=season)
    first_half_start = mlb_dates.first_date_seas[:10]
    first_half_end = mlb_dates.last_date_1sth[:10]
    second_half_start = mlb_dates.first_date_2ndh[:10]
    second_half_end = mlb_dates.last_date_seas[:10]
    return first_half_start, first_half_end, second_half_start, second_half_end

In [ ]:
def get_all_star_break_days(first_half_end, second_half_start):
    start = datetime.strptime(first_half_end, '%Y-%m-%d') + timedelta(days=1)
    stop = datetime.strptime(second_half_start, '%Y-%m-%d') - timedelta(days=1)
    allstar_break_days = list(pd.date_range(start=start, end=stop).astype(str))
    return allstar_break_days

In [ ]:
def get_season_days_so_far(first_half_start, first_half_end, second_half_start, second_half_end):
    today = datetime.today()
    start = datetime.strptime(first_half_start, '%Y-%m-%d')
    season_days = list(pd.date_range(start=start, end=today).astype(str))
    season_days = [d for d in season_days if d not in get_all_star_break_days(first_half_end, second_half_start)]
    return season_days

In [ ]:
this_season = get_current_season()
first_half_start, first_half_end, second_half_start, second_half_end = get_schedule_dates(this_season)
season_days_so_far = get_season_days_so_far(first_half_start, first_half_end, second_half_start, second_half_end)

### Get key for a player

In [ ]:
mlb.playerid_lookup('gallo', 'joey')

In [ ]:
id = int(mlb.playerid_lookup('gallo', 'joey')['key_mlbam'])
print(id)

### Get season-level stats for a all batters

In [ ]:
mlb.batting_stats_bref(season=2019).head() # entire season
mlb.batting_stats_range(start_dt='2019-03-20', end_dt=today) # specified range

#data = mlb.batting_stats_range("2016-08-24",) 
data = mlb.batting_stats_range("2019-05-28", ) 


In [ ]:
df = pd.DataFrame()
df_lst = []

for d in season_days_so_far:
    try:
        tmp_df = mlb.batting_stats_range(str(d), )
        print(' processing schedule date ',d)
    except:
        continue
    tmp_df['DATE'] = d
    df_lst.append(tmp_df)

In [ ]:
df = pd.concat(df_lst)

In [ ]:
df.sort_values(['Name', 'DATE'], inplace=True)

In [ ]:
df

In [ ]:
today = datetime.today()
days = [today - datetime.timedelta(days=x) for x in range(0, numdays)]

In [ ]:
from datetime import date, timedelta

d1 = date(2008, 8, 15)  # start date
d2 = date(2008, 9, 15)  # end date

delta = d2 - d1         # timedelta

for i in range(delta.days + 1):
    print(d1 + timedelta(days=i))



In [ ]:
# Make a list of all days during 2019 baseball season (from beginning to now)
# For each day, pull all data, and append to master dataframe
# data

## Temp

### Get raw stats for a batter

In [ ]:
#gallo_stats = mlb.statcast_batter('2019-04-01', '2019-05-01', id)
gallo_stats = mlb.statcast_batter(start_dt='2019-02-20', end_dt=today, player_id=id)

In [ ]:
gallo_stats[gallo_stats.game_date == '2019-05-01']

In [ ]:
gallo_stats

### Grab all batted ball events for a single day for a single player

In [ ]:
gallo_stats[(gallo_stats.game_date == '2019-05-01') & (~gallo_stats.events.isnull())]

In [ ]:
gallo_stats.events.value_counts()

## What other player sources exist?
* Lahman
** batting stats by year
** updated at the end of each season (current is through 2018)
* fangraps
* baseball reference
* baseball savants


### Do I have to parse the data to one-record-per-game format myself?

### Can I use retrosheet for past years, and use stat_cast for current year?

### How do I define the first game date of the season?

### I should save all player IDs to a table, and append any new players?
